# Predicting FDA Approval of Small Molecule Drugs

## *Metis Project 5 ("Kojak"): Capstone*

### Background
The drug development process is a high-risk, long-term and expensive endeavor. On average, tens of thousands of drug candidates enter the drug discovery pipeline for every one becomes FDA-approved. In addition to this high attrition, this process takes 10 years and is estimated to cost $1.4 billion (1). Machine Learning could help reduce the risk, thereby lowering costs and expediting the process.

<img src="Figures/funnel09.png" alt="Drug Discovery Process" width="800" align=“center”/>

### Data
My data source was ChEMBL: a chemical database curated by the European Molecular Biology Laboratory. It was hosted on Google BigQuery and was also available for download as a PostgreSQL database. It contains information on over 1 million drug-like molecules in 77 tables. I decided to focus on small molecules.

After cleaning and excluding any entries from the past 10 years, I was left with 432,000 entries. My dataset was highly imbalanced, as only 0.4% of these small molecules attained FDA approval.

### Methods

#### *SQL*

I used the Google BigQuery Python Client Library `bigquery` and the helper class `bq_helper` to query the ChEMBL database. I performed a series of SQL queries to extract data from SQL tables into pandas dataframe.

#### *Feature Selection & Engineering*
The ‘COMPOUND_PROPERTIES’ table contained chemical properties of the molecules. These were mostly numerical features and were thus an obvious choice to include. Other tables contained more categorical variables with large numbers of levels. For assays (395 levels) and targets (12,000 levels), I took their assigned parent class. For Target organisms, I created a Boolean variable: was it human or not.

A given small molecule might have multiple activity entries -- in other words, it could have one or more targets and assays associated with it. I thus couldn't simply use `OneHotEncode` on these features, so I used `CountVectorize` to encode this information instead. 

> Before encoding:
    `'UNDEFINED,NON-MOLECULAR,PROTEIN'`

> After CountVectorize:
    

I also tallied the number of activities, assays, targets, and target organisms by molecule. The number of targets and activities were highly correlated, so I merged these into one feature by taking their average. These were also correlated with the number of assays, so I computed the ratio of this average with the number of assays. 

A list of final features is given in my GitHub repository.

#### *Data Processing*

Before sending my data into classifier algorithms, I had to preprocess it as follows: 
-	Impute missing values
-	Scale numerical variables (for Logistic Regression) 
-	CountVectorize the categorical variables for which entries can take on >1 level
-	One-Hot-Encode the other categorical variables
-	engineer features

Because of the diversity of my feature datatypes, I decided to make use of scikit-learn’s Pipelines, using the ColumnTransformer, FunctionTransformer, and FeatureUnion functions. I also created custom TransformerMixin Classes to handle some of the preprocessing tasks.

#### *Modeling*
I evaluated Logistic Regression, Random Forest, Bagged Decision Trees, and “vanilla” Gradient Boosting (Decision Trees) algorithms. These were all out-performed by CatBoost, an algorithm based on gradient-boosted decision trees, which is especially fast and adept at handling categorical variables. I also investigated various methods of over- and under-sampling to address the imbalance in my dataset including SMOTE, ADASYN, Random Undersampling, Edited Nearest Neighbors, Near Miss versions 1 & 3, SMOTE-EEN, SMOTE-Tomek, and assigning class weights. None improved the performance of my models.

#### *Tuning*
I then tuned the hyperparameters of CatBoost. First, I tuned the number of iterations (number of estimators), using F1 score to check for overfitting. (I had to set the p-value low (10-10) to permit the model to fit appropriately. Larger p-values stopped the fitting early and resulted in underfitting.) At a depth of 6, ~175 iterations were optimal. Then I tuned other parameters using BayesSearchCV.